In [ ]:
import torch
from torchvision import datasets, transforms
import numpy as np
import os

digits_per_sequence = 5
n_of_sequences = 60000
output_path = './dataset/'

emnist_dataset = datasets.EMNIST('./EMNIST', split = "digits", train = True, download = True)

dataset_sequences = []
dataset_labels = []
a = []

for i in range(n_of_sequences):
  random_indices = np.random.randint(len(emnist_dataset.data), size=(digits_per_sequence,))
  
  random_digits_images = emnist_dataset.data[random_indices]

  transformed_random_digits_images = []
  
  for img in random_digits_images:
    _temp_img = transforms.ToPILImage()(img)

    # fix EMNIST's transposed images
    _temp_img = transforms.functional.rotate(_temp_img, -90, fill=(0,))
    _temp_img = transforms.functional.hflip(_temp_img)

    # crop pixels
    # _temp_img = transforms.functional.crop(_temp_img, 0, 1, 28, 26)

    # basic augmentation on each EMNIST digit
    _temp_img = transforms.RandomAffine(degrees = 9,  scale=(0.85, 1.1))(_temp_img)
    _temp_img = transforms.ToTensor()(_temp_img).numpy()

    transformed_random_digits_images.append(_temp_img)

  random_digits_images = np.array(transformed_random_digits_images)
  
  random_digits_labels = emnist_dataset.targets[random_indices]

  random_sequence = np.hstack(random_digits_images.reshape(digits_per_sequence, 28, 28))
  random_labels = np.hstack(random_digits_labels.reshape(digits_per_sequence, 1))

  # dataset_sequences.append(random_sequence * 255)
  dataset_sequences.append(random_sequence)
  dataset_labels.append(random_labels)

if not os.path.exists(output_path):
  os.makedirs(output_path)

dataset_data = np.array(dataset_sequences)
dataset_labels = np.array(dataset_labels)

data_path = output_path + "data_" + str(digits_per_sequence) + "_" + str(n_of_sequences) + ".npy"
np.save(data_path, dataset_data)

label_path = output_path + "labels_" + str(digits_per_sequence) + "_" + str(n_of_sequences) + ".npy"
np.save(label_path, dataset_labels)

In [ ]:
import torch.utils.data as data_utils
import numpy as np
import torch

data_path = './dataset/data_5_60000.npy'
labels_path = './dataset/labels_5_60000.npy'

data = np.load(data_path)
data = torch.Tensor(data)
labels = torch.IntTensor(np.load(labels_path).astype(int))

seq_dataset = data_utils.TensorDataset(data, labels)
print(int(len(seq_dataset)))
print(int(len(seq_dataset) * 0.8))
print(int(len(seq_dataset) * 0.2))
train_set, test_set = torch.utils.data.random_split(seq_dataset, [int(len(seq_dataset) * 0.8), int(len(seq_dataset) * 0.2)])

train_size = int(len(seq_dataset) * 0.8)
test_size = int(len(seq_dataset) * 0.2)

train_loader = torch.utils.data.DataLoader(train_set, batch_size = train_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = test_size, shuffle = True)

In [ ]:
import matplotlib.pyplot as plt

number_of_printed_imgs = 5

for batch_id, (x, y) in enumerate(train_loader):
  for j in range(len(x)):
    plt.imshow(x[j], cmap='gray')
    plt.show()

    print(y[j])
    number_of_printed_imgs -= 1

    if number_of_printed_imgs <= 0:
      break

  if number_of_printed_imgs <= 0:
      break

In [ ]:
x_train = []
y_train = []

for batchId, (i, j) in enumerate(train_loader):
  x_train = i.numpy()
  y_train = j.numpy()

In [ ]:
x_test = []
y_test = []

for batchId, (i, j) in enumerate(test_loader):
  x_test = i.numpy()
  y_test = j.numpy()

In [ ]:
del(train_loader)
del(test_loader)

In [ ]:
x_test.shape, x_train.shape

In [ ]:
x_test[0].shape

In [ ]:
train_croped = []
for i in range(x_train.shape[0]):
    a = x_train[i]
    for j in range(x_train.shape[2]):
        if j >= a.shape[1]:
            continue
        else:
            if np.all(a[:, j] <= 0.2):
                a = np.delete(a, j, 1)
                j -= 1

    train_croped.append(a)

In [ ]:
test_croped = []
for i in range(test_size):
    a = x_test[i]
    for j in range(x_test.shape[2]):
        if j >= a.shape[1]:
            continue
        else:
            if np.all(a[:, j] <= 0.2):
                a = np.delete(a, j, 1)
                j -= 1

    test_croped.append(a)

In [ ]:
del(x_train)
del(x_test)

In [ ]:
max = 0
z = 0

for i in range(train_size):
    if train_croped[i].shape[1] > max:
        max = train_croped[i].shape[1]
        z = i
print(max)

In [ ]:
max = 0
z = 0

for i in range(test_size):
    if test_croped[i].shape[1] > max:
        max = test_croped[i].shape[1]
        z = i
print(max)

In [ ]:
min = 1000
z = 0

for i in range(train_size):
    if train_croped[i].shape[1] < min:
        min = train_croped[i].shape[1]
        z = i
print(min)
ind_train = z

In [ ]:
min = 1000
z = 0

for i in range(test_size):
    if test_croped[i].shape[1] < min:
        min = test_croped[i].shape[1]
        z = i
print(min)
ind_test = z

In [ ]:
for i in range(len(train_croped)):
    train_croped[i] = train_croped[i] * 255

In [ ]:
for i in range(len(test_croped)):
    test_croped[i] = test_croped[i] * 255

In [ ]:
from PIL import Image as im

for i in range(len(train_croped)):
    dummy = np.array(train_croped[i], dtype=np.uint8)
    data = im.fromarray(dummy)
    data.save('./train/tr%d.jpg' % (i))

In [ ]:
from PIL import Image as im

for i in range(len(test_croped)):
    dummy = np.array(test_croped[i], dtype=np.uint8)
    data = im.fromarray(dummy)
    data.save('./test/te%d.jpg' % (i))

In [ ]:
for i in range(len(train_croped)):
    image = im.open('./train/tr%d.jpg' % (i))
    new_image = image.resize((140, 28))
    new_image.save('./train_reshaped/tr%d.jpg' % (i))

In [ ]:
for i in range(len(test_croped)):
    image = im.open('./test/te%d.jpg' % (i))
    new_image = image.resize((140, 28))
    new_image.save('./test_reshaped/te%d.jpg' % (i))

In [ ]:
X_train = []

for i in range(len(train_croped)):
    o = im.open('./train_reshaped/tr%d.jpg' % (i))
    o_arr = np.array(o)
    o_arr = o_arr / 255
    X_train.append(o_arr)

In [ ]:
X_test = []

for i in range(len(test_croped)):
    o = im.open('./test_reshaped/te%d.jpg' % (i))
    o_arr = np.array(o)
    o_arr = o_arr / 255
    X_test.append(o_arr)

In [ ]:
X_train = np.array(X_train)

X_train.shape

In [ ]:
X_test = np.array(X_test)

X_test.shape

In [ ]:
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)

np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)